In [4]:
!pip install accelerate bitsandbytes langchain chromadb safetensors kaleido python-multipart transformers sentence-transformers pypdf openai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=2f9917c2fc378f81bea70a5853535a95f8f8971e0066360c51556c84c7d663f1
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [5]:
import transformers
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'
model_name = "ybelkada/fuyu-8b-sharded"
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [6]:
import torch
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.llms import HuggingFacePipeline

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit = True,
    torch_dtype = torch.bfloat16,
    quantization_config = bnb_config,
    device_map = 'auto'
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.bos_token_id = 1
pipe = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    max_length = 2048,
    temperature = 0,
    top_p = 0.95,
    repetition_penalty = 1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/58.2k [00:00<?, ?B/s]

pytorch_model-00001-of-00011.bin:   0%|          | 0.00/2.15G [00:00<?, ?B/s]

pytorch_model-00002-of-00011.bin:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

pytorch_model-00003-of-00011.bin:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/848 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/6.13M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

In [9]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

loader = PyPDFLoader("/content/cs224u-contextualreps-2023-handout.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150,
)

splits = text_splitter.split_documents(docs)

embedding = embed_model

vectordb = Chroma.from_documents(
    documents = splits,
    embedding = embedding,
)

In [10]:
local_llm("What are the statical vector representations of words?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:71013 for open-end generation.


'\n\n1. Color: The representation of color is represented by shades of blue, green or yellow.\n2. Gesture: The gesture of the characters represents their body movements and gestures. In this case, the characters are holding hands while walking together.\n3. Space: The space between the characters and the background represents the environment in which the scene is set. In the image, there is a bench and a street nearby.\n4. Movements: The movement of the characters and their body language represent the dynamic nature of the situation they are experiencing. The image shows two people engaged in physical activity, possibly dancing or conversing with each other.\n5. Interaction: The interaction between the two characters can be inferred from the visual cues like touching hands and sharing the space between them. This interaction highlights the social aspect of this scene.'

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
import os

os.environ["OPENAI_API_KEY"] = "api-key"

llm = ChatOpenAI(model_name= "gpt-3.5-turbo-0301", temperature= 0)

template = """Given the following context answer the question at the end. If you don't know the answer return can't answer
{context}
Question: {question}
Answer:"""
qa_chain_prompt = PromptTemplate.from_template(template)

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = vectordb.as_retriever(),
    chain_type_kwargs = {"prompt": qa_chain_prompt} # You can choose whatever chaintype you want
)

In [ ]:
question = "What are the statical vecoter representations of words?"

result = qa_chain({"query": question})
result["result"]